In [132]:
from matplotlib import image
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import openpyxl
import xlsxwriter
import statsmodels.api as sm
import cv2
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from PIL import Image, ImageFilter

current_path = os.getcwd()
print(current_path)

C:\Users\lbv06


In [133]:
# B_10_real ... B_16_real
image = cv2.imread(r"d:\usuarios\lbv06\Downloads\Abasalt_Microscope_68hr\Analysis\B_15_real.jpg")

sens_x = 50
sens_y = 60
scale_percent = 50 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)

In [134]:
# https://stackoverflow.com/questions/58751101/count-number-of-cells-in-the-image
# and have a look at here: https://www.geeksforgeeks.org/white-and-black-dot-detection-using-opencv-python/


imgContours = image.copy()

imgGray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
imgGray = imgGray[:, :]

imgBlur = cv2.GaussianBlur(imgGray, (5,5), 1)
imgCanny = cv2.Canny(imgBlur, sens_x, sens_y)    # with these parameters I can change the sensitivity to dot size


#thresh = cv2.threshold(imgGray, 100, 255, cv2.THRESH_BINARY)[1]
thresh = cv2.threshold(imgGray, 100, 300, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

img_contours, _ = cv2.findContours(imgCanny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contDraw = cv2.drawContours(imgContours, img_contours, -1, (255, 0, 255), 1)
cv2.putText(contDraw, "Number of dots: {}".format(len(img_contours)), (40, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 1)

print("\nNumber of dots: {}".format(len(img_contours)))


thresh1 = cv2.resize(thresh, dim, interpolation=cv2.INTER_AREA)
image1 = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
contDraw1 = cv2.resize(contDraw, dim, interpolation=cv2.INTER_AREA)



cv2.imshow("Theshold", thresh1)
Hori = np.concatenate((image1, contDraw1), axis=1) 
cv2.imshow("Original image vs. Contour image", Hori)

cv2.waitKey(0)
cv2.destroyAllWindows()


Number of dots: 963


In [136]:
# try 3


# Deep copy for results:
inputImageCopy = image.copy()

# Convert RGB to grayscale:
grayscaleImage = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Find the contours on the binary image:

imgBlur = cv2.GaussianBlur(grayscaleImage, (5,5), 1)
imgCanny = cv2.Canny(imgBlur, sens_x, sens_y)    # with these parameters I can change the sensitivity to dot size


thresh = cv2.threshold(grayscaleImage, 100, 300, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

contours, hierarchy = cv2.findContours(imgCanny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
#contDraw = cv2.drawContours(inputImageCopy, contours, -1, (255, 0, 255), 1)

# Store bounding rectangles and object id here:
objectData = []

# ObjectCounter:
objectCounter = 1

# Look for the outer bounding boxes (no children):
for _, c in enumerate(contours):
    # Get the contour's bounding rectangle:
    boundRect = cv2.boundingRect(c)

    # Store in list:
    objectData.append((objectCounter, boundRect))

    # Get the dimensions of the bounding rect:
    rectX = boundRect[0]
    rectY = boundRect[1]
    rectWidth = boundRect[2]
    rectHeight = boundRect[3]

    # Draw bounding rect:
    color = (0, 0, 255)
    cv2.rectangle(inputImageCopy, (int(rectX), int(rectY)),
                  (int(rectX + rectWidth), int(rectY + rectHeight)), color, 2)

    # Draw object counter:
    font = cv2.FONT_HERSHEY_SIMPLEX
    fontScale = 1
    fontThickness = 1
    color = (0, 0, 255)
#    cv2.putText(inputImageCopy, str(objectCounter), (int(rectX), int(rectY)), font, fontScale, color, fontThickness)

    # Increment object counter
    objectCounter += 1

objectCounter -= 1
print("\nNumber of dots: {}".format(objectCounter))


inputImageCopy1 = cv2.resize(inputImageCopy, dim, interpolation=cv2.INTER_AREA)
cv2.imshow("Rectangles", inputImageCopy1)


cv2.waitKey(3000)
cv2.destroyAllWindows()




Number of dots: 963
